# Homework 3

## Question 1

#### Consider the following table which details the use of contraception by age of currently married women in El Salvador in 1985.

<table>
  <tr>
    <th></th>
    <th colspan="3">Contraceptive Method</th>
  </tr>
  <tr>
    <td><b>Age</b></td>
    <td><b>Ster.</b></td>
    <td><b>Other</b></td>
    <td><b>None</b></td>
  </tr>
  <tr>
    <td>15 - 19</td>
    <td>3</td>
    <td>61</td>
    <td>232</td>
  </tr>
  <tr>
    <td>20 - 24</td>
    <td>80</td>
    <td>137</td>
    <td>400</td>
  </tr>
  <tr>
    <td>25 - 29</td>
    <td>216</td>
    <td>131</td>
    <td>301</td>
  </tr>
  <tr>
    <td>30 - 34</td>
    <td>268</td>
    <td>76</td>
    <td>203</td>
  </tr>
  <tr>
    <td>35 - 39</td>
    <td>197</td>
    <td>50</td>
    <td>188</td>
  </tr>
  <tr>
    <td>40 - 44</td>
    <td>150</td>
    <td>24</td>
    <td>164</td>
  </tr>
  <tr>
    <td>45 - 49</td>
    <td>91</td>
    <td>10</td>
    <td>183</td>
  </tr>
</table>

#### Let $ t_{i} $ denote an indicator variable which takes on values $ \{1,2,3\} $ where 1 is Sterilization, 2 is Other, and 3 is None. Consider the activation function:

$$ a_{ij}=\alpha_{j}+\beta_{j}x_{i}+\gamma_{j}x_{i}^{2} $$

#### where $ x_{i} $ is the mid-point of the $ i $-th age group, and $ j \in \{1,2,3\} $. Recall that the multinomial logit model is given by

$$ p\left(t_{i}=j|x_{i}\right)=\frac{exp\left(a_{ij}\right)}{\sum_{j'}exp\left(a_{ij'}\right)} $$

#### Implement multinomial logistic regression from scratch (for the underlying optimization you may use modules such as the ones available in $ \displaystyle{scipy} $) and apply it to the given data. Comment on your observations.

In [ ]:
from scipy.special import factorial
from scipy.optimize import minimize
import numpy as np
import matplotlib.pyplot as plt

x = [17, 22, 27, 32, 37, 42, 47]

M = np.matrix([[  3,  62, 232],
               [ 80, 137, 400],
               [216, 131, 301],
               [268,  76, 203],
               [197,  50, 188],
               [150,  24, 164],
               [ 91,  10, 183]])

def m(i, j):
    return M[i - 1, j - 1]

def activation(w, i, j):
    s_idx = (j - 1) * 3
    return w[s_idx] + w[s_idx + 1] * x[i - 1] + w[s_idx + 2] * x[i - 1]**2

c_logit_sum = {}

def logit(w, i, j):
    key = str(w) + '_' + str(i)
    if key not in c_logit_sum:
        c_logit_sum[key] = np.exp(activation(w, i, 1)) + np.exp(activation(w, i, 2)) + np.exp(activation(w, i, 3))
    sum = c_logit_sum[key]
    return np.exp(activation(w, i, j)) / sum

def error(w, i):
    return - m(i, 1) * np.log(logit(w, i, 1)) - m(i, 2) * np.log(logit(w, i, 2)) - m(i, 3) * np.log(logit(w, i, 3))

w0 = [100, 100, 100, 100, 100, 100, 100, 100, 100]
# w_ml = []
# for s in range(len(x)):
#     res = minimize(neg_likelihood, w0, args=(s + 1, ), method='COBYLA')
#     if (res.success):
#         w_ml.append(res.x)

# print(w_ml)

res = minimize(error, w0, args=(4, ), method='COBYLA')
if (res.success):
    print(res.x)
else:
    print(res)


# w1_ml = [-4.30243814e+00, -7.60004802e-04,  1.21589921e-07,
#         -1.29240916e+00,  3.29655268e-04,  1.35854539e-08,
#          3.28688361e-02, -3.78431790e-07, -2.09425802e-07]

# print(logit(w_ml, 5, 1))
# print(logit(w_ml, 5, 2))
# print(logit(w_ml, 5, 3))

## Question 2

#### Download the Enron-spam dataset from http://www.aueb.gr/users/ion/data/enron-spam/. Use the pre-processed datasets Enron1, . . . , Enron5 as training data and Enron6 as test data.

**Implement the Naive Bayes algorithm that we discussed in the class (a good in-depth reference is http://nlp.stanford.edu/IR-book/pdf/13bayes.pdf). Remember to perform all calculations on the log scale to prevent underow.**

- ** Report the accuracy on the test set.**

- **How do you account for dierent prior probabilities for spam and ham?**

- **Does the performance of the classier change when you do add one Laplace smoothing?**

- **What are the most discriminative words as per the naive Bayes classifer?**

We first extract all messages from archives.

In [1]:
import tarfile
import glob

WORKDIR = 'workdir'

In [2]:
def msg_dir(members):
    for tarinfo in members:
        if (tarinfo.name.count('/') > 1):
            yield tarinfo

for file in glob.glob("Enron-Spam/*.tar.gz"):
    print("Extracting " + file)
    with tarfile.open(file, 'r:gz') as tar:
        tar.extractall(path=WORKDIR, members=msg_dir(tar))

Extracting Enron-Spam/enron6.tar.gz
Extracting Enron-Spam/enron5.tar.gz
Extracting Enron-Spam/enron2.tar.gz
Extracting Enron-Spam/enron1.tar.gz
Extracting Enron-Spam/enron4.tar.gz
Extracting Enron-Spam/enron3.tar.gz


Then, start to train the Naive Bayes classifier.

> 1) Define a function which strip meaningless words, which includes punctuation mark and stopwords. We get the "Default English stopwords" from http://www.ranks.nl/stopwords

In [3]:
from string import punctuation
import re

punc_regex = re.compile('[%s]' % re.escape(punctuation.replace("'", "")))
apos_regex = re.compile('(\w)\s\'\s(\w)')

STOPWORDS_FILENAME = 'stopwords.txt'
with open(STOPWORDS_FILENAME, 'r') as f:
    stopwords = set(f.read().splitlines())

# strip meaningless words from a string and return a
# meaningful list of words.
def strip_meaningless(s):
    # replace any punctuation marks but the apostrophe with space
    clean_s = punc_regex.sub(' ', s)
    
    # format case "what ' s" to "what's" in order to capture the stopwrods like "that's"
    clean_s = apos_regex.sub('\\1\'\\2', clean_s)
    
    c_l = [word.lower() for word in clean_s.split()]
    return [word for word in c_l if word not in stopwords]

>  2) Extract all vocabulary and count the number of message in class ham and spam, respectively, in this round.

In [4]:
num_hams = 0
num_spams = 0

words_ham = []
words_spam = []

FILE_ENCODING = 'ISO-8859-1'

# As the question mentioned, use the pre-processed datasets
# Enron1, ..., Enron5 as training data and Enron6 as test data.
for file in glob.glob(WORKDIR + '/enron[1-5]/*/*.txt'):
    is_ham = file.endswith('ham.txt')
    if is_ham:
        num_hams += 1
    else:
        num_spams += 1

    with open(file, 'r', encoding=FILE_ENCODING) as f_msg:
        content = strip_meaningless(f_msg.read())
        
        if is_ham:
            words_ham.extend(content)
        else:
            words_spam.extend(content)

vocabulary = set(words_ham + words_spam)
vocabulary.remove("'")    # we think the apostrophe should not be one of word in vocabulary

> Save all the vocabulary to vocabulary.txt if you want to have a look.

In [5]:
with open('out/vocabulary.txt', mode='w', encoding=FILE_ENCODING) as voc_file:
    voc_file.write('\n'.join(vocabulary))

> 3) Compute $ \hat{P}\left(c\right) $ and all $ \hat{P}\left(t_{k}|c\right) $

In [6]:
import operator

prior_ham = num_hams * 1.0 / (num_hams + num_spams)
prior_spam = num_spams * 1.0 / (num_hams + num_spams)

print("Prior probabilities for ham: " + str(prior_ham))
print("Prior probabilities for spam: " + str(prior_spam))

words_ham_count = {}
for word in words_ham:
    if word in words_ham_count:
        words_ham_count[word] += 1
    else:
        words_ham_count[word] = 1

words_spam_count = {}
for word in words_spam:
    if word in words_spam_count:
        words_spam_count[word] += 1
    else:
        words_spam_count[word] = 1

condprob_ham = {}
condprob_spam = {}

for word in vocabulary:
    condprob_ham[word] = (words_ham_count.get(word, 0) + 1) * 1.0 / (len(words_ham) + len(vocabulary))
    condprob_spam[word] = (words_spam_count.get(word, 0) + 1) * 1.0 / (len(words_spam) + len(vocabulary))
    
    # without Laplace smoothing
#     condprob_ham[word] = words_ham_count.get(word, 0) * 1.0 / len(words_ham)
#     condprob_spam[word] = words_spam_count.get(word, 0) * 1.0 / len(words_spam)

# sort condprob_ham and condprob_spam by values
sorted_condprob_ham = sorted(condprob_ham.items(), key=operator.itemgetter(1))
sorted_condprob_spam = sorted(condprob_spam.items(), key=operator.itemgetter(1))

print("The most discriminative words in condprob_ham: " + str(sorted_condprob_ham[-1]))
print("The most discriminative words in condprob_spam: " + str(sorted_condprob_spam[-1]))

Prior probabilities for ham: 0.5428272477991052
Prior probabilities for spam: 0.4571727522008948
The most discriminative words in condprob_ham: ('enron', 0.016913688602462673)
The most discriminative words in condprob_spam: ('subject', 0.007590890505477282)


> 4) Now we can apply this Naive Bayes classifier to do prediction.

In [7]:
import numpy as np

corrects  = 0
num_test_msgs = 0

for file in glob.glob(WORKDIR + '/enron6/*/*.txt'):
    is_true_ham = file.endswith('ham.txt')
    
    with open(file, 'r', encoding=FILE_ENCODING) as f_msg:
        content = strip_meaningless(f_msg.read())
        
        score_ham = np.log(prior_ham)
        score_spam = np.log(prior_spam)
        for word in content:
            if word in vocabulary:    # ignore word if it's not in the vocabulary
                score_ham += np.log(condprob_ham[word])
                score_spam += np.log(condprob_spam[word])
            
        num_test_msgs += 1
        
        if ((score_ham > score_spam) and is_true_ham) or ((score_ham < score_spam) and not is_true_ham):
            corrects += 1

print("The accuracy on the test set is: " + str(corrects  * 1.0 / num_test_msgs))

The accuracy on the test set is: 0.9818333333333333


Finally, we clean all temporary files.

In [8]:
import shutil

shutil.rmtree(WORKDIR)

# Question 3

#### Work on the same dataset as the above problem.

- **Implement the binary logistic regression algorithm that we discussed in the class. You may use any optimizer of your choice including optimization modules from $ \displaystyle{scipy} $.**
 - **Report accuracy on test set.**
 - **How do you account for different prior probabilities for spam and ham?**
 - **What are the most discriminative words as per the logistic regression classifer?**

- **Implement the Bayesian logistic regression algorithm that we discussed in the class.**
 - **Report accuracy on test set.**
 - **How does the accuracy change when the strength of the prior changes?**

#### Comment on the differences and similarities between the three classifers that you implemented to solve the spam detection problem (naive bayes, binary logistic regression and bayesian logistic regression)?